In [ ]:
# The pre-cleaned data can be downloaded from my github
!git clone --depth 1 https://github.com/rfcoffee/Buffalo-Niagara-Employment/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib notebook

In [ ]:
# read CSV to dataframe and simple formatting
cpi = pd.read_csv('Buffalo-Niagara-Employment/cpi_northeast.csv')
emp = pd.read_csv('Buffalo-Niagara-Employment/emp_tot.csv')
hmean = pd.read_csv('Buffalo-Niagara-Employment/h_mean.csv')

hmean = hmean.fillna(method='ffill')
cpi['Time'] = list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'), cpi['Time']))
emp['date'] = list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'), emp['date']))
emp.set_index('date', inplace=True)
hmean['date'] = list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'), hmean['date']))
hmean.set_index('date', inplace=True)

cpi.drop('Unnamed: 0', axis=1, inplace=True)
cpi = cpi.rename(columns={'Time': 'date'})
cpi.set_index('date',inplace=True)
cpi = cpi.loc[emp.index.values]

In [ ]:
import matplotlib.animation as animation
job = emp.columns.values
x = emp.index.values

In [ ]:
# create the function that will do the plotting, where curr is the current frame
def update(curr):
    if curr == (len(job)-1): 
        a.event_source.stop()
    plt.clf()
    plt.gcf().text(0.013, 0.75, 'Estimated Total Employment', fontsize=10, rotation='vertical', color='blue')
    plt.gcf().text(0.976, 0.7, 'Mean Hourly Wage / CPI', fontsize=10, rotation='vertical', color='red')
    
    ax = plt.gca()
    ax2 = ax.twinx()
    ax.tick_params(axis='y', which='major', labelsize=8)
    ax2.tick_params(axis='y', which='major', labelsize=8)
    
    ax.axis([datetime.datetime(1999,1,1,0,0),datetime.datetime(2018,1,1,0,0),min(emp[job[curr]])/1.1,1.1*max(emp[job[curr]])])
    ax.plot(x, emp[job[curr]], 'bo-')
    ax.set_xlabel('Time')
    ax.tick_params('y', colors='b')
    plt.gcf().text(0.2, 0.91, 'Occupation:  '+job[curr], ha='left')
    for item in ax.xaxis.get_ticklabels():
        item.set_rotation(45)
    
    ax2.plot(x, hmean[job[curr]]/cpi['Cpi'], 'ro-')
    ax2.tick_params('y', colors='r')


# save emp & h_mean variation of all occupations to animation
fig = plt.figure(figsize=(7,5))
fig.subplots_adjust(bottom=0.2, left = 0.125, right=0.875, top=0.9)
a = animation.FuncAnimation(fig, update,interval=700, save_count=21)#, blit=True) frames=90,

In [ ]:
# save file to mp4. mp4 can be converted to gif using linux command ffmpeg
# note: turn off matplotlib inline before using save()
# a.save('movie.mp4', dpi=500)